# Write sum code

Write a function, in the programming language of your choice, which finds contiguous regions (or "islands") in a matrix where all values in the island are greater than a threshold (but not necessarily the same). 

The function should take a threshold, a minimum island size, and an arbitrarily sized matrix as inputs.   
The function should output a matrix (same size as the input matrix) of booleans. 

Do not wrap around matrix edges. Corner neighbors are not sufficient for island continuity. 

For example, if the the inputs are:  
- threshold = 5  
- min island size = 3  
- matrix = [
        4, 4, 4, 2, 2;  
        4, 2, 2, 2, 2;  
        2, 2, 8, 7, 2;  
        2, 8, 8, 8, 2;  
        8, 2, 2, 2, 8  
    ].  

Then the output would be  
- islands = [
        0, 0, 0, 0, 0;  
        0, 0, 0, 0, 0;  
        0, 0, 1, 1, 0;  
        0, 1, 1, 1, 0;  
        0, 0, 0, 0, 0
    ].

In [85]:
import numpy as np

In [86]:
def find_islands(threshold, min_island_size, matrix):
    '''Finds contiguous regions (or "islands") in a matrix where all values in the island are greater than a threshold (but not necessarily the same). 
    This function does not wrap around matrix edges, and corner neighbors are not sufficient for island continuity.
    Assumes args are reasonably typed.

    Args:
        threshold: the number which all values in an island must be greater than
        min_island_size: the number of values greater than the threshold which must be present in a contiguous region to be considered an island
        matrix: an arbitrarily sized matrix
        
    Returns:
        islands: a boolean matrix (same size as the input matrix) that represents the "islands" found
    '''
    
    if type(matrix) is not np.array:
        matrix = np.asarray(matrix)
    
    indices_of_elements_greater_than_threshold = list(zip(*np.where(matrix > threshold)))
    
    def find_index_neighbors(index):
        '''Given an index and a list of indices, finds all neighboring indices. The index may be in the list. 

        Args:
            index: tuple of ints that represents the index of an element in a matrix: (row, column) 
            [defined above, not passed] indices_of_elements_greater_than_threshold: 
                list of indices that match description of an index above: [(i_0, j_0, ..., (i_n, j_n)]
        Returns:
            index_neighbors: a list of index neighbors, also represented by a tuple of ints
        '''
        row, column = index

        index_neighbors = [] 

        for i, j in indices_of_elements_greater_than_threshold:
            # How far away are the two indices?
            index_difference = (abs(row-i), abs(column-j))

            # To be neighbors, one of the row or column must be the same (index difference of 0) 
            # and one must be +/-1 (index difference of 1). It doesn't matter which, so we can check those together.
            if sum(index_difference) == 1:
                index_neighbors += [(i, j)]
  
        return index_neighbors
    
    def find_contiguous_region(current_index, contiguous_region_indices):
        '''Explores indices to find all indices in a region contiguous with current_index. 
        
        Args:
            current_index: the index (as described in find_index_neighbors) from which to find all contiguous indices
            contiguous_region_indices [used for recursive calls -- should be an empty list in a non-recursive call]: 
                a list of the indices that have already been found to be contiguous with current_index
        Returns:
            contiguous_region_indices: a list of all indices that are contiguous with current_index
        '''
        
        # Add current index
        contiguous_region_indices += [current_index]
        
        # Add unexplored index_neighbors, and recursively, their unexplored index_neighbors 
        # Base case occurs when no new index_neighbors are found 
        for index_neighbor in find_index_neighbors(current_index):
            if index_neighbor not in contiguous_region_indices:

                new_contiguous_indices = [
                    new_contiguous_index 
                    for new_contiguous_index in find_contiguous_region(index_neighbor, contiguous_region_indices) 
                    if new_contiguous_index not in contiguous_region_indices
                ]
                contiguous_region_indices += new_contiguous_indices
         
        return contiguous_region_indices
    
    # The key will be the element index, and the value will be whether the element is within an island
    # If the key does not exist, the index has not been "explored" yet
    is_index_in_an_island = {} 
    
    for index in indices_of_elements_greater_than_threshold:
        if index not in is_index_in_an_island:
            potential_island_indices = find_contiguous_region(index, [])
            
            # Add potential_island if it is sufficiently large
            for island_index in potential_island_indices:
                is_index_in_an_island[island_index] = len(potential_island_indices) >= min_island_size
    
    # Chose the most readable option from https://stackoverflow.com/questions/56660387/fastest-way-to-convert-a-list-of-indices-to-2d-numpy-array-of-ones
    island_indices = [index for index, is_island in is_index_in_an_island.items() if is_island]
    
    islands = np.zeros(matrix.shape, int)
    for i in island_indices:
        islands[i] = 1
 
    return islands
           

In [87]:
A=np.array([
    [4, 4, 4, 2, 2],  
    [4, 2, 2, 2, 2],  
    [2, 2, 8, 7, 2],  
    [2, 8, 8, 8, 2],  
    [8, 2, 2, 2, 8],  
])

find_islands(threshold=5, min_island_size=3, matrix=A)

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]])

## Things I learned during this exercise:
A python default argument is defined when the function is defined, and not every time the function is called: https://stackoverflow.com/questions/16549768/lifetime-of-default-function-arguments-in-python

## What sensors and actuators would you expect a modern, small, geostationary, telecommunication satellite to have?

## For GNC

### Sensors
- GNSS
- Star trackers
- Earth sensor, sun sensor
- Gyroscopes
- Accelerometers (linear & angular)
- Pressure/drag sensors (not as familiar with these)
- Is a clock a sensor? I would say so.

### Actuators
- Reaction wheels
- Electric propulsion thrusters (and ideally, no chemical prop thrusters because they're heavy)
- Solar Array Drive Assemblies (SADA)

## Other sensors/systems
- RF -- receivers/antennae 
- Thermal -- thermistors, thermocouples (basically everywhere!)
- Power -- solar array input, susbsystem output, battery charge
- Pressure -- for propulsion management